In [1]:
!which python

/home/t_kim/.virtualenvs/dev-python3.7/bin/python


In [2]:
import torch
from fairseq.data.data_utils import collate_tokens

roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')
roberta.eval()

batch_of_pairs = [
    ['Roberta is a heavily optimized version of BERT.', 'Roberta is not very optimized.'],
    ['Roberta is a heavily optimized version of BERT.', 'Roberta is based on BERT.'],
    ['potatoes are awesome.', 'I like to run.'],
    ['Mars is very far from earth.', 'Mars is very close.'],
]

batch = collate_tokens(
    [roberta.encode(pair[0], pair[1]) for pair in batch_of_pairs], pad_idx=1
)

logprobs = roberta.predict('mnli', batch)
print(logprobs.argmax(dim=1))

Using cache found in /home/t_kim/.cache/torch/hub/pytorch_fairseq_master
100%|██████████| 751652118/751652118 [01:05<00:00, 11520216.77B/s]


tensor([0, 2, 1, 0])


In [5]:
import torch.nn as nn

In [64]:
logprobs.argmax(dim=1)

tensor([0, 2, 1, 0])

In [65]:
logprobs.detach().cpu().numpy().argmax(dim=1)

TypeError: 'dim' is an invalid keyword argument for argmax()

In [62]:
np.exp(logprobs.detach().cpu().numpy())

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [37]:
for lp, l in zip(logprobs, [0,2,1,0]):
    cel = -lp[l]
    print(cel)

tensor(0.0016, grad_fn=<NegBackward>)
tensor(0.0192, grad_fn=<NegBackward>)
tensor(0.7050, grad_fn=<NegBackward>)
tensor(0.0060, grad_fn=<NegBackward>)


In [39]:
cel.detach().numpy()

array(0.00595225, dtype=float32)

In [38]:
for lp, l in zip(logprobs, [0,2,1,0]):
    cel = -lp[0]
    print(cel)

tensor(0.0016, grad_fn=<NegBackward>)
tensor(6.4762, grad_fn=<NegBackward>)
tensor(1.2907, grad_fn=<NegBackward>)
tensor(0.0060, grad_fn=<NegBackward>)


In [26]:
import numpy as np
probs = np.exp(logprobs.detach().numpy())

In [66]:
probs

array([[9.9836308e-01, 9.9690806e-04, 6.4006459e-04],
       [1.5396120e-03, 1.7428910e-02, 9.8103154e-01],
       [2.7508199e-01, 4.9408698e-01, 2.3083103e-01],
       [9.9406546e-01, 2.5547303e-03, 3.3797924e-03]], dtype=float32)

In [34]:
for p, l in zip(probs, [0,2,1,0]):
    print(p, l)
#     nn.CrossEntropyLoss()(p, l)

[9.983631e-01 9.969081e-04 6.400646e-04] 0
[0.00153961 0.01742891 0.98103154] 2
[0.275082   0.49408698 0.23083103] 1
[0.99406546 0.00255473 0.00337979] 0


In [31]:
probs

array([[9.9836308e-01, 9.9690806e-04, 6.4006459e-04],
       [1.5396120e-03, 1.7428910e-02, 9.8103154e-01],
       [2.7508199e-01, 4.9408698e-01, 2.3083103e-01],
       [9.9406546e-01, 2.5547303e-03, 3.3797924e-03]], dtype=float32)

In [15]:
nn.Softmax(dim=1)(logprobs)

tensor([[9.9836e-01, 9.9691e-04, 6.4006e-04],
        [1.5396e-03, 1.7429e-02, 9.8103e-01],
        [2.7508e-01, 4.9409e-01, 2.3083e-01],
        [9.9407e-01, 2.5547e-03, 3.3798e-03]], grad_fn=<SoftmaxBackward>)

In [14]:
nn.Softmax(dim=1)(logprobs).detach().numpy().round(3)

array([[0.998, 0.001, 0.001],
       [0.002, 0.017, 0.981],
       [0.275, 0.494, 0.231],
       [0.994, 0.003, 0.003]], dtype=float32)

In [40]:
def list_duplicates_of(seq,item):
    start_at = -1
    locs = []
    while True:
        try:
            loc = seq.index(item,start_at+1)
        except ValueError:
            break
        else:
            locs.append(loc)
            start_at = loc
    return locs

source = "ABABDBAAEDSBQEWBAFLSAFB"
print(list_duplicates_of(source, 'B'))

[1, 3, 5, 11, 15, 22]


In [55]:
with open('/home/t_kim/repos/erc/Datasets/MELD/roberta/test.input1', 'r') as stream:
    lines = [line.strip() for line in stream.readlines()]

foo = 0
for utterance in set(lines):
    print(list_duplicates_of(lines, utterance))
    foo += len(list_duplicates_of(lines, utterance))

[4800, 4801, 4802, 4803]
[8657, 8658, 8659, 8660]
[7338, 7339, 7340, 7341, 7342, 7343]
[3556, 3557, 3558]
[13914, 13915]
[934, 935, 936, 937, 938, 939, 940, 941, 942, 943]
[2840, 2841, 2842, 2843, 2844]
[9062, 9063, 9064, 9065, 9066]
[13421, 13422, 13423, 13424, 13425, 13426, 13427]
[9809, 9810]
[2923, 2924, 2925]
[3993, 3994]
[13515, 13516]
[12572, 12573]
[10695, 10696]
[15134, 15135, 15136, 15137, 15138, 15139, 15140, 15141, 15142, 15143, 15144]
[3640, 3641, 3642, 3643, 3644, 3645, 3646, 3647, 3648, 3649, 3650, 3651]
[14314, 14315, 14316, 14317, 14318, 14319, 14320, 14321, 14322, 14323, 14324, 14325, 14326]
[129, 130, 131, 132, 133, 134, 135]
[5237, 5238, 5239, 5240, 5241, 5242, 5243, 5244, 16671]
[14648, 14649]
[7627, 7628, 7629, 7630, 7631, 7632, 7633, 7634, 7635, 7636, 7637, 7638, 7639, 7640, 7641, 7642, 7643, 7644]
[9715, 9716]
[6705, 6706, 6707, 6708, 6709, 6710, 6711, 6712]
[350, 351, 352, 353, 354, 355, 356, 357, 358, 359]
[6102, 6103, 6104, 6105]
[11469, 11470, 11471, 11472, 

[3250, 3251, 3252, 3253, 3254, 3255, 3256, 3257]
[5035, 5036, 5037, 5038, 5039, 5040, 5041, 5042, 5043]
[1495, 1496, 1497, 1498, 1499, 1500, 1501, 1502]
[8247, 8248, 8249, 8250, 8251, 8252]
[3306, 3307, 3308, 3309]
[16270, 16271, 16272, 16273, 16274, 16275]
[3189, 3190, 3191, 3192, 3193, 3194, 3195, 3196, 3197, 3198, 3199, 3200, 3201, 3202, 3203, 3204, 3205]
[7766, 7767, 7768, 7769, 7770, 7771, 7772, 7773, 7774, 7775, 7776, 7777]
[3476, 3477]
[3665, 3666, 3667, 3668, 3669, 3670, 3671, 3672, 3673, 3674, 3675, 3676, 3677, 3678]
[10594]
[11652, 11653, 11654, 11655, 11656, 11657, 11658, 11659, 11660, 11661, 11662, 11663, 11664, 11665, 11666, 11667, 11668, 11669]
[13285, 13286, 13287, 13288, 13289, 13290, 13291, 13292, 13293]
[14362, 14363, 14364, 14365, 14366, 14367, 14368]
[12664, 12665, 12666]
[15790, 15791, 15792, 15793, 15794, 15795, 15796, 15797, 15798, 15799, 15800, 15801, 15802, 15803, 15804, 15805, 15806, 15807, 15808, 15809, 15810]
[14490]
[15107, 15108, 15109, 15110, 15111, 15112

[5219, 5220, 5221, 5222, 5223]
[16963, 16964, 16965, 16966, 16967, 16968, 16969, 16970, 16971, 16972]
[5955, 5956, 5957, 5958, 5959, 5960]
[2821, 2822]
[7368]
[13712, 13713]
[17184, 17185, 17186, 17187, 17188, 17189, 17190, 17191, 17192, 17193]
[16071, 16072, 16073, 16074]
[2295, 2296, 2297, 2298, 2299, 2300, 2301, 2302, 2303, 2304]
[12510, 12511, 12512, 12513, 12514, 12515, 12516, 12517, 12518, 12519, 12520, 12521, 12522, 12523, 12524]
[476]
[4143, 4144, 4145, 4146]
[10496, 10497]
[1903, 1904, 1905, 1906, 1907]
[8450, 8451, 8452, 8453]
[15646, 15647, 15648, 15649, 15650, 15651, 15652, 15653, 15654, 15655, 15656, 15657]
[14176, 14177, 14178, 14179, 14180, 14181, 14182, 14183, 14184, 14185]
[17858, 17859, 17860, 17861, 17862, 17863, 17864, 17865]
[16482, 16483, 16484, 16485, 16486, 16487, 16488, 16489, 16490, 16491, 16492, 16493, 16494, 16495, 16496, 16497, 16498]
[8233, 8234]
[14219, 14220]
[7404, 7405, 7406, 7407, 7408, 7409, 7410, 7411, 7412]
[311, 312, 313, 314]
[14797]
[11565, 1156

[890, 891]
[13481, 13482, 13483]
[3779, 3780, 3781, 3782]
[840, 841, 842, 843]
[17009, 17010, 17011, 17012, 17013, 17014, 17015, 17016, 17017, 17018, 17019, 17020, 17021, 17022]
[6033, 6034, 6035, 6036, 6037, 6038]
[2061, 2062, 2063, 2064, 2065, 2066]
[17230, 17231, 17232, 17233, 17234, 17235, 17236, 17237, 17238, 17239, 17240, 17241, 17242, 17243]
[6968, 6969, 6970, 6971, 6972, 6973, 6974, 6975, 6976, 6977, 6978, 6979, 6980, 6981, 6982, 6983, 6984, 6985, 6986]
[7266, 7267, 7268, 7269, 7270, 7271, 7272]
[11924, 11925, 11926, 11927, 11928, 11929, 11930, 11931, 11932, 11933]
[3419, 3420, 3421, 3422, 3423, 3424, 3425, 3426]
[12553, 12554, 12555, 12556, 12557, 12558, 12559, 12560]
[13880, 13881, 13882]
[13887, 13888, 13889, 13890, 13891]
[835, 836]
[5198, 5199, 5200, 5201, 5202]
[11860, 11861, 11862, 11863, 11864, 11865, 11866, 11867, 11868, 11869, 11870, 11871, 11872, 11873, 11874, 11875, 11876, 11877, 11878]
[12586, 12587, 12588, 12589, 12590, 12591]
[191, 192, 193, 194, 195]
[1431, 1432

[1893]
[3028, 3029, 3030]
[17493, 17494, 17495, 17496, 17497, 17498]
[7701, 7702]
[16065]
[17604, 17605, 17606, 17607]
[3981, 3982, 3983, 3984, 3985, 3986, 3987, 3988, 3989, 3990, 3991]
[15590, 15591, 15592, 15593, 15594]
[1409, 1410, 1411, 1412]
[4095, 4096, 4097, 4098, 4099, 4100, 4101, 4102, 4103, 4104, 4105, 4106, 4107]
[9939, 9940, 9941, 9942, 9943, 9944, 9945, 9946]
[14728, 14729, 14730, 14731, 14732, 14733, 14734, 14735, 14736]
[7502, 7503, 7504, 7505, 7506, 7507, 7508, 7509]
[3081, 3082, 3083, 3084, 3085, 3086, 3087, 3088]
[13717, 13718, 13719]
[9415, 9416, 9417]
[7791, 7792, 7793, 7794, 7795, 7796, 7797, 7798, 7799, 7800, 7801, 7802, 7803, 7804]
[4788, 4789, 4790, 4791, 4792, 4793]
[10445, 10446, 10447, 10448, 10449, 10450, 10451, 10452, 10453, 10454, 10455]
[11780, 11781, 11782, 11783, 11784, 11785, 11786, 11787, 11788, 11789, 11790, 11791, 11792, 11793]
[10573]
[13892, 13893, 13894, 13895, 13896, 13897]
[2878, 2879, 2880]
[12852, 12853, 12854, 12855, 12856, 12857, 12858, 128

In [56]:
len(lines), len(set(lines))

(17966, 2509)

In [57]:
foo

17966